## convolutional layer with pooling

Pooling is a technique often used with convolutional layers.
This is explored in this notebook. The model is configurated to test this.

In [1]:
from os.path import join

raw = join('data', 'raw')
processed = join('data', 'processed')

from src.training_env import reset_and_populate

reset_and_populate(raw, processed, [400,0,100])

['data\\processed\\train\\n',
 'data\\processed\\validation\\n',
 'data\\processed\\test\\n',
 'data\\processed\\train\\o',
 'data\\processed\\validation\\o',
 'data\\processed\\test\\o',
 'data\\processed\\train\\x',
 'data\\processed\\validation\\x',
 'data\\processed\\test\\x']

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def create_generator(data_dir, batch_size, datagen):
    full_path = join(processed, data_dir)
    return datagen.flow_from_directory(
        full_path,
        target_size=(32, 32),
        color_mode='grayscale',
        batch_size=batch_size,
        class_mode='binary')

train_datagen = ImageDataGenerator(
        rescale = 1./255,
        rotation_range=360,
        horizontal_flip=True,
        vertical_flip=True)

test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = create_generator('train', 20, train_datagen)
test_generator = create_generator('test', 10, test_datagen)

Found 1200 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


The model is slightly changed:
Another convolutional layer is inserted and two `MaxPooling2D` layers are added. Those have a kernel size of two by two, which is halving the subsequent layer in width and height, resulting in much smaller layers. Please note that the `padding='same'` is removed, which has no particular reason at this point.

The model has again less parameters than the model before.

In [3]:
from tensorflow.keras import layers
from tensorflow.keras import models

model = models.Sequential()
model.add(layers.Conv2D(8, (2,2), activation='relu', input_shape=(32, 32, 1)))
model.add(layers.Conv2D(8, (2,2), activation='relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(8, (2,2), activation='relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Flatten())
model.add(layers.Dense(32,'relu'))
model.add(layers.Dense(3, 'softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 31, 31, 8)         40        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 8)         264       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 8)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 8)         264      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 8)           0         
_________________________________________________________________
flatten (Flatten)            (None, 392)               0         
_________________________________________________________________
dense (Dense)                (None, 32)                12

In [4]:
from tensorflow.keras.optimizers import SGD

optimizer = SGD(lr=0.005, momentum=0.9, nesterov=True)

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['acc'])

In [5]:
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
from datetime import datetime
from os import mkdir

log_dir = join('logs', 'srp641', datetime.now().strftime("%Y-%m-%dT%H-%M-%S"))
mkdir(log_dir)

from src.training_env import reset
reset(log_dir)

callbacks = [ TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    embeddings_freq=1) ]

history = model.fit_generator(
    train_generator,
    steps_per_epoch=20,
    epochs=20,
    callbacks=callbacks)

Epoch 1/20
20/20 [==============================] - 2s 106ms/step - loss: 1.0947 - acc: 0.3350
Epoch 2/20
20/20 [==============================] - 1s 60ms/step - loss: 1.0727 - acc: 0.4675
Epoch 3/20
20/20 [==============================] - 1s 60ms/step - loss: 1.0299 - acc: 0.4725
Epoch 4/20
20/20 [==============================] - 1s 54ms/step - loss: 0.9696 - acc: 0.6350
Epoch 5/20
20/20 [==============================] - 1s 51ms/step - loss: 0.8780 - acc: 0.7325
Epoch 6/20
20/20 [==============================] - 1s 50ms/step - loss: 0.8448 - acc: 0.6975
Epoch 7/20
20/20 [==============================] - 1s 53ms/step - loss: 0.6947 - acc: 0.7550
Epoch 8/20
20/20 [==============================] - 1s 51ms/step - loss: 0.5979 - acc: 0.7925
Epoch 9/20
20/20 [==============================] - 1s 52ms/step - loss: 0.5840 - acc: 0.7825
Epoch 10/20
20/20 [==============================] - 1s 52ms/step - loss: 0.4754 - acc: 0.8425
Epoch 11/20
20/20 [==============================] - 1s 50

In [6]:
model.evaluate_generator(test_generator)

[0.30849321695665516, 0.91333336]

Adding pooling layer to the model again increased the accuracy by reducing the size of the model (in this instance: 232kb to 149kb).

In [7]:
model_path = join('models', 'devel', 'srp641.h5')
model.save(model_path)